In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import seaborn as sns
import sys 
import urllib
import urllib.request
import joblib

In [ ]:
df=pd.read_csv("archive/dataset.csv")
df1=pd.read_csv("archive/Symptom-severity.csv")

In [ ]:
df.shape

In [ ]:
df.head()


In [ ]:
df.drop("Disease",axis="columns",inplace=True)
df.head()

In [ ]:
df[df["Symptom_1"]=="itching"]

In [ ]:
# print("Total Unique symptoms:" , pd.concat([df["Symptom_1"], df["Symptom_2"],df["Symptom_3"],df["Symptom_4"],df["Symptom_5"],df["Symptom_6"],df["Symptom_7"],df["Symptom_8"],df["Symptom_9"],df["Symptom_10"],df["Symptom_11"],df["Symptom_12"],df["Symptom_13"],df["Symptom_14"],df["Symptom_15"],df["Symptom_16"],df["Symptom_17"]]).unique().__len__())

In [ ]:
#to check all the null values in all the column
df.isna().sum()
df.isnull().sum()

In [ ]:
#As we can see above, Onwards 6 col the no. of null values are increasing.

In [ ]:
#Cleaning Data

cols=df.columns  #df.columns return all the columns name/head present in dataset
data = df[cols].values.flatten() #Basically flatten the 2d array into 1d array.
s = pd.Series(data)
s = s.str.strip() #removes any unwanted white space
s = s.values.reshape(df.shape) # converts the 1d array into the data shape
df = pd.DataFrame(s, columns=df.columns) #modifying original dataset with new cleaned data
df = df.fillna(0) #fills 0 in place of null 
df.tail() #displays data in from tail.

In [ ]:
#In section we will encode each symptoms with its severity.
#For each symptoms we have weight in the df1 dataset, and we will perform and encoding operation to map its value.

vals = df.values  #storing data into vals.
symptoms = df1['Symptom'].unique() # storing all the available unique symptoms from 2nd dataset

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]

d = pd.DataFrame(vals,columns=cols) #creates a new dataset where vals=New encoded symptoms value & cols=og column head/name 

#Since in df1 we don't have values for symptoms dischromic _patches,foul_smell_of urine and spotting_ urination
#we will put 0 inplace of them


d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination',0)
df = d.replace('foul_smell_of urine',0) #replacing and storing in original df

df.head()

In [ ]:
# Now lets have a look at the different symptoms, we will need this list for option inputs in front-end
symptoms

In [ ]:
(df[cols] == 0).all()

In [ ]:
df['Disease'].value_counts()

In [ ]:
# These may be referenced later from front end

df['Disease'].unique()

In [ ]:
#This is the X param of our model.fit(X,Y)
data = df.iloc[:,1:].values
data

In [ ]:
#This is the Y param of our model(X,Y)
labels = df['Disease'].values
labels

In [ ]:
# Train Test split is done from the dataset
x_train, x_test, y_train, y_test = train_test_split(data, labels, shuffle=True, train_size = 0.85)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
model = SVC()
#model.fit(x_train, y_train)

In [ ]:
model.fit(data,labels)
joblib.dump(model,'medicine-recommender.joblib')

In [ ]:
x_test

In [ ]:
preds = model.predict(x_test)
preds

In [ ]:
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('F1-score% =', f1_score(y_test, preds, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, preds)*100)
sns.heatmap(df_cm)

In [ ]:
model.predict([[4,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])